In [3]:
import os
from pathlib import Path
from operator import itemgetter
import pickle
import pydicom
from time import time
import pandas
import traceback

import matplotlib.pyplot as plt
import numpy as np
import cv2
from shapely.geometry import Polygon, MultiPolygon, LineString, GeometryCollection, Point, MultiPoint
from shapely.affinity import translate

from catch_converter.parse_contours import parse_cvi42ws
from LazyLuna.Mini_LL import *
from LazyLuna.utils import *
from LazyLuna.Figures import *
from LazyLuna.Views import *

In [6]:
# basepaths
bp        = '/Users/dietrichhadler/Desktop/Daten/CAMAID'
bp_annos  = '/Users/dietrichhadler/Desktop/Daten/CAMAID/Gold'
bp_cases  = '/Users/dietrichhadler/Desktop/Daten/CAMAID/Cases'
bp_imgs   = '/Users/dietrichhadler/Desktop/Daten/CAMAID/Reader2'

In [ ]:
parse_cvi42ws(bp_annos, bp_annos, process=True, debug=False)

In [7]:
lax_cine_view = LAX_CINE_View()
cases = []

# get the paths of imgs and annotations via SeriesInstanceUIDs
paths = get_imgs_and_annotation_paths(bp_imgs, bp_annos)
print(paths)
for imgp, annop in paths:
    #if not '1.3.12.2.1107.5.2.43.167071.30000021082405410237400000007' in annop: continue
    try:
        print(imgp, annop)
        st = time()
        case = Case(imgp, annop, os.path.basename(imgp), os.path.basename(bp_annos))
        case = lax_cine_view.initialize_case(case)
        case.store(bp_cases)
        cases.append(case)
        print('Case customization took: ', time()-st, 'Case: ', case.reader_name, case.case_name)
    except Exception as e:
        print('Failed at: ', imgp, annop, ' , Exception: ', e)
        print(traceback.format_exc())
    print()

[('/Users/dietrichhadler/Desktop/Daten/Atria_Intra/Imgs/Atria_HCM_24_', '/Users/dietrichhadler/Desktop/Daten/Atria_Intra/HaOne/1.3.6.1.4.1.53684.1.1.2.1955809981.5808.1655726131.43120'), ('/Users/dietrichhadler/Desktop/Daten/Atria_Intra/Imgs/Atria_COV_31_', '/Users/dietrichhadler/Desktop/Daten/Atria_Intra/HaOne/1.3.6.1.4.1.53684.1.1.2.1955809981.5808.1655727316.84791'), ('/Users/dietrichhadler/Desktop/Daten/Atria_Intra/Imgs/Atria_VOL_17_', '/Users/dietrichhadler/Desktop/Daten/Atria_Intra/HaOne/1.3.6.1.4.1.53684.1.1.2.1955809981.4572.1657189397.957'), ('/Users/dietrichhadler/Desktop/Daten/Atria_Intra/Imgs/Atria_COV_32_', '/Users/dietrichhadler/Desktop/Daten/Atria_Intra/HaOne/1.3.6.1.4.1.53684.1.1.2.1955809981.5768.1657006519.2009'), ('/Users/dietrichhadler/Desktop/Daten/Atria_Intra/Imgs/Atria_MD_4_', '/Users/dietrichhadler/Desktop/Daten/Atria_Intra/HaOne/1.3.6.1.4.1.53684.1.1.2.1955809981.5808.1655724961.5617'), ('/Users/dietrichhadler/Desktop/Daten/Atria_Intra/Imgs/Atria_PSORCOR_40_', 

Case customization took:  5.054916858673096 Case:  HaOne Atria_MD_12_

/Users/dietrichhadler/Desktop/Daten/Atria_Intra/Imgs/Atria_PSORCOR_36_(1) /Users/dietrichhadler/Desktop/Daten/Atria_Intra/HaOne/1.3.6.1.4.1.53684.1.1.2.1955809981.5768.1657013877.5314
Case customization took:  14.276689052581787 Case:  HaOne Atria_PSORCOR_36_(1)

/Users/dietrichhadler/Desktop/Daten/Atria_Intra/Imgs/Atria_MD_11_ /Users/dietrichhadler/Desktop/Daten/Atria_Intra/HaOne/1.3.6.1.4.1.53684.1.1.2.1955809981.5808.1655725239.16642
Case customization took:  4.723749160766602 Case:  HaOne Atria_MD_11_

/Users/dietrichhadler/Desktop/Daten/Atria_Intra/Imgs/Atria_PSORCOR_37_ /Users/dietrichhadler/Desktop/Daten/Atria_Intra/HaOne/1.3.6.1.4.1.53684.1.1.2.1955809981.5808.1655727514.88438
Case customization took:  10.116650104522705 Case:  HaOne Atria_PSORCOR_37_

/Users/dietrichhadler/Desktop/Daten/Atria_Intra/Imgs/Atria_MD_13_ /Users/dietrichhadler/Desktop/Daten/Atria_Intra/HaOne/1.3.6.1.4.1.53684.1.1.2.1955809981.580

In [7]:
cases = [pickle.load(open(os.path.join(bp_cases, p), 'rb')) for p in os.listdir(bp_cases)]
view  = SAX_T1_View()
#cases = [view.customize_case(c) for c in cases if c.reader_name=='AI']
cases_tmp = []
for c in cases:
    try:
        cases_tmp.append(view.customize_case(c))
    except Exception as e: 
        print('Error: ', traceback.print_exc())
        continue
cases = cases_tmp
print('Nr Cases: ', len(cases))

Error:  None
Error:  None
Error:  None
Error:  None
Error:  None
Error:  None
Nr Cases:  24


Traceback (most recent call last):
  File "<ipython-input-7-45fc11e6b597>", line 7, in <module>
    cases_tmp.append(view.customize_case(c))
  File "/Users/dietrichhadler/Documents/GitHub/LazyLuna/src/LazyLuna/Views.py", line 460, in customize_case
    else: case.attach_categories([SAX_T1_Category])
  File "/Users/dietrichhadler/Documents/GitHub/LazyLuna/src/LazyLuna/Mini_LL.py", line 265, in attach_categories
    for c in categories: self.categories.append(c(self))
  File "/Users/dietrichhadler/Documents/GitHub/LazyLuna/src/LazyLuna/Categories.py", line 495, in __init__
    self.sop2depthandtime = self.get_sop2depthandtime(case.imgs_sop2filepath, debug=debug)
  File "/Users/dietrichhadler/Documents/GitHub/LazyLuna/src/LazyLuna/Categories.py", line 520, in get_sop2depthandtime
    img1, img2 = imgs[depthandtime2sop[(0,0)]], imgs[depthandtime2sop[(1,0)]]
KeyError: (1, 0)
Traceback (most recent call last):
  File "<ipython-input-7-45fc11e6b597>", line 7, in <module>
    cases_tmp.append(

In [10]:
cases1 = sorted([c for c in cases if c.reader_name=='Gold'], key=lambda c:c.case_name)
cases2 = sorted([c for c in cases if c.reader_name=='Reader2'], key=lambda c:c.case_name)
#cases1 = [cases1[0]]+cases1[2:]
for i in range(max(len(cases1),len(cases2))):
    print(cases1[i].case_name, cases2[i].case_name)
print(len(cases1))
print(len(cases2))
ccs = [Case_Comparison(c1,c2) for c1,c2 in zip(cases1,cases2)]
print(len(ccs))

CAMAID_001_1 CAMAID_001_1
CAMAID_002_1 CAMAID_002_1
CAMAID_003_1 CAMAID_003_1
Camaid_004_1 Camaid_004_1
Camaid_006_1 Camaid_006_1
Camaid_007_1 Camaid_007_1
Camaid_008_1 Camaid_008_1
Camaid_009_1 Camaid_009_1
Camaid_010_1 Camaid_010_1
Camaid_011_1 Camaid_011_1
Camaid_014_1 Camaid_014_1
Camaid_017_1 Camaid_017_1
12
12
12


In [ ]:
cols = ['case name', '']
rows = []

for cc in ccs:
    